# Building a Model With sklearn - Lending Club

## whats this?

In [129]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Import necessary libraries

In [130]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import warnings
import gc
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
%matplotlib inline

from sklearn.impute  import SimpleImputer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [131]:
pd.set_option('display.max_columns',60)

## Load Dataset

In [132]:
start_df = pd.read_csv('./loan.csv', low_memory=False)

Work with a copy of dataset to avoid having to reload:

In [133]:
df = start_df.copy(deep=True)
df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,...,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,...,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,...,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,...,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,...,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,...,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Define target variable

Based on problem defined for dataset, we are building a model that will predict whether a user will defaul in future.

Therefore, we need to convert the historical loan status categories into a boolean that describes wheter or not the customer defaulted. 

In [134]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

Defines which 'loan status' is considered as 'Defaulted':

In [135]:
target_list = [1 if i in ['Charged Off', 'Does not meet the credit policy. Status:Charged Off', 'Default','Late (31-120 days)','Late (16-30 days)'] else 0 for i in df['loan_status']]

Calculate how much of dataset 'Defaults' based on this description:

In [136]:
print(sum(target_list), sum(target_list)/len(target_list))

61176 0.06894010338310913


In [137]:
df['TARGET'] = target_list

In [141]:
df['issue_d'].unique()

array(['Dec-2011', 'Nov-2011', 'Oct-2011', 'Sep-2011', 'Aug-2011',
       'Jul-2011', 'Jun-2011', 'May-2011', 'Apr-2011', 'Mar-2011',
       'Feb-2011', 'Jan-2011', 'Dec-2010', 'Nov-2010', 'Oct-2010',
       'Sep-2010', 'Aug-2010', 'Jul-2010', 'Jun-2010', 'May-2010',
       'Apr-2010', 'Mar-2010', 'Feb-2010', 'Jan-2010', 'Dec-2009',
       'Nov-2009', 'Oct-2009', 'Sep-2009', 'Aug-2009', 'Jul-2009',
       'Jun-2009', 'May-2009', 'Apr-2009', 'Mar-2009', 'Feb-2009',
       'Jan-2009', 'Dec-2008', 'Nov-2008', 'Oct-2008', 'Sep-2008',
       'Aug-2008', 'Jul-2008', 'Jun-2008', 'May-2008', 'Apr-2008',
       'Mar-2008', 'Feb-2008', 'Jan-2008', 'Dec-2007', 'Nov-2007',
       'Oct-2007', 'Sep-2007', 'Aug-2007', 'Jul-2007', 'Jun-2007',
       'Dec-2013', 'Nov-2013', 'Oct-2013', 'Sep-2013', 'Aug-2013',
       'Jul-2013', 'Jun-2013', 'May-2013', 'Apr-2013', 'Mar-2013',
       'Feb-2013', 'Jan-2013', 'Dec-2012', 'Nov-2012', 'Oct-2012',
       'Sep-2012', 'Aug-2012', 'Jul-2012', 'Jun-2012', 'May-20

## Sample Last 5 Years of Data

In [143]:
df['issue_d'] = pd.to_datetime(df['issue_d'])
df['year'] = df['issue_d'].dt.year

In [144]:
df = df[df.year >= 2010]

## Handle Missing Values

In [ ]:
print(df.isnull().values.any())
sns.heatmap(df.isnull()) 

True


In [ ]:
def null_values(df, perct_to_remove = 100):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        columns_to_remove = mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values'] > perct_to_remove].index
        columns_to_keep = df.iloc[:,~df.columns.isin(columns_to_remove)].columns
        return mis_val_table_ren_columns , columns_to_keep 

In [ ]:
cols_missing_values, col_to_keep = null_values(df , 10)

In [ ]:
df = df[col_to_keep]
nullCount = df.isnull().sum()
nullCount[nullCount>0]

In [ ]:
# Number of each type of column
df.dtypes.value_counts().sort_values().plot(kind='barh')
plt.title('Number of columns distributed by Data Types',fontsize=20)
plt.xlabel('Number of columns',fontsize=15)
plt.ylabel('Data type',fontsize=15)

## Removing Columns that have more than 10 categories

In [ ]:
## Helper function to easily get columns with specific data type
def get_specific_columns(df, data_types, to_ignore = list(), ignore_target = False):
    columns = df.select_dtypes(include=data_types).columns
    if ignore_target:
        columns = filter(lambda x: x not in to_ignore, list(columns))
    return columns

In [ ]:
## Save all columns with 'object' datatype (non-numerical)
obj_variables = get_specific_columns(df, ['object'], ['TARGET'], ignore_target = True)
obj = df[list(obj_variables)]

In [ ]:
not_to_much_unique = df.loc[:,~df.columns.isin(obj.loc[:,obj.nunique() > 10].columns)].columns

In [ ]:
df = df[not_to_much_unique]

In [ ]:
 ### why are we doing this?

In [ ]:
df.drop(['collections_12_mths_ex_med' , 'tot_coll_amt' ,'id', 'loan_status' ] , axis = 1 , inplace = True)

## Impute missing values

In [ ]:
nullCount = df.isnull().sum()
nullCount[nullCount>0]

In [ ]:
#### since 5 columns have the same nulls we decided to drop these 29 rows 
df = df.loc[~df[ 'delinq_2yrs' ].isnull()]

In [ ]:
### here we use SimpleImputer form sklearn to impute NA, the strategy decided is median to not change the distribution since tehy are skewed
imp = SimpleImputer( strategy='median', verbose=1)
numericals = list(get_specific_columns(df, ['int64' , 'float64'], ['TARGET'], ignore_target = True))

num = df[numericals]

df[numericals] =imp.fit_transform(num) 

## Feature Engineering

In [ ]:
# 1 
intrst_per_grade = df[['int_rate','grade']].groupby('grade').agg(['mean','std'])
dum = df[['grade','int_rate']].join(intrst_per_grade , on ='grade')
dum['intrst_per_grade'] = (dum['int_rate'] - dum[('int_rate', 'mean')]) / dum[('int_rate', 'std')]
df['intrst_per_grade']= dum['intrst_per_grade']

In [ ]:
# 2 %  last_paymnt_amnt / tot_cur_bal
df['perc_last_pay_from_bal'] = df['installment'] / (df['tot_cur_bal'] + 1)

In [ ]:
# 3 [last_pay - ( tot_cur_bal + annual_income/12)] if positive  1(dsnt have wnough money to pay) if negative has 0 
df['not_engh_to_pay'] = df['installment'] - df['tot_cur_bal'] - df['annual_inc']/12
df['not_engh_to_pay'] = np.where(df['not_engh_to_pay'] > 0, 1, 0).astype('object')

## Scaling Numerical Values

In [ ]:
scaler = StandardScaler()
numericals = list(get_specific_columns(df, ['int64' , 'float64'], ['TARGET' , 'member_id'], ignore_target = True))
num = df[numericals]

df[numericals] =scaler.fit_transform(num)

## Binnng

In [ ]:
skt = KBinsDiscretizer(n_bins = 5 , strategy = 'quantile' , encode = 'ordinal')
to_bin = np.array(df['total_pymnt']).reshape(-1, 1)
df[['total_pymnt']] = skt.fit_transform(to_bin)
df[['total_pymnt']] = df[['total_pymnt']].astype('object' , inplace =True)

In [ ]:
## whats this?

In [ ]:
#### interest rate compared to ppl in ur grade 
#### %  last_paymnt_amnt / tot_cur_bal
####   [last_pay - ( tot_cur_bal + annual_income/12)] if positive  1(dsnt have wnough money to pay) if negative has 0 

## One Hot Encoding

In [ ]:
def One_hot_sklearn(df , categoricals): 
    df.copy()
    for i in categoricals:
        print(i)
        hot = OneHotEncoder()
        X = hot.fit_transform(df[i].values.reshape(-1,1)).toarray()
        dfOneHot = pd.DataFrame(X, columns = [str(i)+str('_')+str(j[3:]) for j in hot.get_feature_names()])
        dfOneHot['member_id'] = df['member_id']
        df = df.merge(dfOneHot,on = 'member_id')
        df.drop(i, axis =1 , inplace = True)
    return df 

In [ ]:
obj_variables = list(get_specific_columns(df, ['object'], ['TARGET'], ignore_target = True))
df = One_hot_sklearn(df , obj_variables)

In [ ]:
df.head(10)

## Splitting Train and Test Set

In [ ]:
X = df.loc[:,df.columns!='TARGET']
y = df['TARGET']

In [ ]:
## change this to split based on dates

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, shuffle = False, random_state=100)

## Modelling

#### Logistic Regression

In [ ]:
log = LogisticRegression()

In [ ]:
log.fit(X_train, y_train)

In [ ]:
predLog = log.predict(X_test)

In [ ]:
accuracy_score(y_test, predLog)

In [ ]:
f1_score(y_test, predLog)

#### Random Forest Regressor

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train,y_train)

In [ ]:
predRF = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, predRF)

In [ ]:
f1_score(y_test, predRF)

## Cross Validation and Hyperparameter Tuning

In [ ]:
## Perform grid search with multiple parameter options for Random Forest Classifier

In [ ]:
# Number of trees in random forest
n_estimators = [50, 100, 200]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [100, 150]
# Minimum number of samples required to split a node
min_samples_split = [2, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
kfold = KFold(n_splits=3)

# grid_search = GridSearchCV(
#         estimator=RandomForestClassifier(),
#         param_grid=random_grid,
# #         scoring="accuracy",
#         scoring = make_scorer(f1_score)
#         cv=kfold,
#         refit=True,
#         n_jobs=-1,
#         verbose = 2,
# )

grid_search = RandomizedSearchCV(
        estimator=RandomForestClassifier(),
        param_distributions=random_grid,
#         scoring="accuracy",
        scoring = make_scorer(f1_score),
        cv=kfold,
        refit=True,
        random_state=999,
        n_iter = 1,
        n_jobs=-1,
        verbose = 2,
)

grid_result = grid_search.fit(X_test, y_test)

In [ ]:
print(
    "\tAverage result for best Random Forest: {} +/- {:.5f}".format(
        grid_result.best_score_,
        grid_result.cv_results_["std_test_score"][
            np.argmax(grid_result.cv_results_["mean_test_score"])
        ],
    )
)

In [ ]:
grid_result.best_params_

## Metrics

#### ROC Curve (Random Forest Regressor)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, predRF)
print(logit_roc_auc)
fpr, tpr, thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
plt.figure()  
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show();